In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
import time
from shapely.geometry import MultiPolygon
import pickle


In [20]:
file = 'local_pkl/about_df_dict_by_map_2025-02-07.pkl'
file_goget_json = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/gem-tracker-maps/trackers/goget/compilation_output/Oil & Gas Extraction-map-file-2025-02-20.csv'

In [21]:
# df = pickle.load(file)
df = gpd.read_file(file_goget_json)

In [25]:

# country_list = set(df['country-list'].to_list())
# print(country_list)

countries = set(df['country/area'].apply(lambda x: x.split('-')).apply(lambda x: '; '.join(x)).to_list())
print(sorted(countries))

# missing = country_list - countries


# print(missing)

['Albania', 'Algeria', 'Angola', 'Angola; Republic of the Congo', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Barbados', 'Bolivia', 'Brazil', 'Brunei', 'Cameroon', 'Canada', 'Chad', 'Chile', 'China', 'China; Japan', 'Colombia', 'Cuba', 'Cyprus', "Côte d'Ivoire", 'Denmark', 'Ecuador', 'Egypt', 'Ethiopia', 'France', 'Gabon', 'Germany', 'Ghana', 'Grenada', 'Guatemala', 'Guyana', 'Hungary', 'India', 'Indonesia', 'Iran', 'Iran; Iraq', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Kazakhstan', 'Kenya', 'Kuwait', 'Kuwait; Saudi Arabia', 'Kuwait; Saudi Arabia; Iran', 'Libya', 'Madagascar', 'Malaysia', 'Mauritania', 'Mexico', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Netherlands', 'New Zealand', 'Nigeria', 'Norway', 'Oman', 'Pakistan', 'Palestine', 'Papua New Guinea', 'Peru', 'Philippines', 'Poland', 'Qatar', 'Republic of the Congo', 'Romania', 'Russia', 'Russia; Kazakhstan', 'Saudi Arabia', 'Saudi Arabia; Bahrain', 'Saudi Arabia; Iran', 'Sene

In [ ]:
current_countries = {
  "Africa": [
    "Algeria",
    "Angola",
    "Angola-Republic of the Congo",
    "Benin",
    "Botswana",
    "British Indian Ocean Territory",
    "Burkina Faso",
    "Burundi",
    "Cabo Verde",
    "Cameroon",
    "Central African Republic",
    "Chad",
    "Comoros",
    "Côte d'Ivoire",
    "Djibouti",
    "DR Congo",
    "Egypt",
    "Equatorial Guinea",
    "Eritrea",
    "Eswatini",
    "Ethiopia",
    "French Southern Territories",
    "Gabon",
    "The Gambia",
    "Ghana",
    "Guinea",
    "Guinea-Bissau",
    "Kenya",
    "Lesotho",
    "Liberia",
    "Libya",
    "Madagascar",
    "Malawi",
    "Mali",
    "Mauritania",
    "Mauritius",
    "Mayotte",
    "Morocco",
    "Mozambique",
    "Namibia",
    "Niger",
    "Nigeria",
    "Republic of the Congo",
    "Réunion",
    "Rwanda",
    "Saint Helena, Ascension, and Tristan da Cunha",
    "Sao Tome and Principe",
    "Senegal",
    "Senegal-Mauritania",
    "Seychelles",
    "Sierra Leone",
    "Somalia",
    "South Africa",
    "South Sudan",
    "Sudan",
    "Tanzania",
    "Togo",
    "Tunisia",
    "Uganda",
    "Western Sahara",
    "Zambia",
    "Zimbabwe"
  ],
  "Americas": [
    "Anguilla",
    "Antigua and Barbuda",
    "Argentina",
    "Aruba",
    "Bahamas",
    "Barbados",
    "Belize",
    "Bermuda",
    "Bolivia",
    "Bonaire, Sint Eustatius, and Saba",
    "Bouvet Island",
    "Brazil",
    "Canada",
    "Cayman Islands",
    "Chile",
    "Colombia",
    "Costa Rica",
    "Cuba",
    "Curaçao",
    "Dominica",
    "Dominican Republic",
    "Ecuador",
    "El Salvador",
    "Falkland Islands",
    "French Guiana",
    "Greenland",
    "Grenada",
    "Guadeloupe",
    "Guatemala",
    "Guyana",
    "Haiti",
    "Honduras",
    "Jamaica",
    "Martinique",
    "Mexico",
    "Montserrat",
    "Nicaragua",
    "Panama",
    "Paraguay",
    "Peru",
    "Puerto Rico",
    "Saint Barthélemy",
    "Saint Kitts and Nevis",
    "Saint Lucia",
    "Saint Martin (French part)",
    "Saint Pierre and Miquelon",
    "Saint Vincent and the Grenadines",
    "Sint Maarten (Dutch part)",
    "South Georgia and the South Sandwich Islands",
    "Suriname",
    "Trinidad and Tobago",
    "Turks and Caicos Islands",
    "United States",
    "Uruguay",
    "Venezuela",
    "Virgin Islands (British)",
    "Virgin Islands (U.S.)"
  ],
  "Asia": [
    "Afghanistan",
    "Armenia",
    "Azerbaijan",
    "Azerbaijan-Turkmenistan",
    "Bahrain",
    "Bangladesh",
    "Bhutan",
    "Brunei",
    "Cambodia",
    "China",
    "China-Japan",
    "Cyprus",
    "Georgia",
    "Hong Kong",
    "India",
    "Indonesia",
    "Iran",
    "Iran-Iraq",
    "Iraq",
    "Israel",
    "Japan",
    "Jordan",
    "Kazakhstan",
    "Kuwait",
    "Kuwait-Saudi Arabia",
    "Kuwait-Saudi Arabia-Iran",
    "Kyrgyzstan",
    "Laos",
    "Lebanon",
    "Macao",
    "Malaysia",
    "Maldives",
    "Mongolia",
    "Myanmar",
    "Nepal",
    "North Korea",
    "Oman",
    "Pakistan",
    "Palestine",
    "Philippines",
    "Qatar",
    "Russia-Kazakhstan",
    "Saudi Arabia",
    "Saudi Arabia-Bahrain",
    "Saudi Arabia-Iran",
    "Singapore",
    "South China Sea",
    "South Korea",
    "Sri Lanka",
    "Syria",
    "Taiwan",
    "Tajikistan",
    "Thailand",
    "Thailand-Malaysia",
    "Timor-Leste",
    "Türkiye",
    "Turkmenistan",
    "United Arab Emirates",
    "United Arab Emirates-Iran",
    "Uzbekistan",
    "Vietnam",
    "Vietnam-Malaysia",
    "Yemen"
  ],
  "Europe": [
    "Åland Islands",
    "Albania",
    "Andorra",
    "Austria",
    "Belarus",
    "Belgium",
    "Bosnia and Herzegovina",
    "Bulgaria",
    "Croatia",
    "Czech Republic",
    "Denmark",
    "Estonia",
    "Faroe Islands",
    "Finland",
    "France",
    "Germany",
    "Gibraltar",
    "Greece",
    "Guernsey",
    "Holy See",
    "Hungary",
    "Iceland",
    "Ireland",
    "Isle of Man",
    "Italy",
    "Jersey",
    "Kosovo",
    "Latvia",
    "Liechtenstein",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Moldova",
    "Monaco",
    "Montenegro",
    "Netherlands",
    "North Macedonia",
    "Norway",
    "Poland",
    "Portugal",
    "Romania",
    "Russia",
    "Russia-Kazakhstan",
    "San Marino",
    "Serbia",
    "Slovakia",
    "Slovenia",
    "Spain",
    "Svalbard and Jan Mayen",
    "Sweden",
    "Switzerland",
    "Ukraine",
    "United Kingdom"
  ],
  "Oceania": [
    "American Samoa",
    "Australia",
    "Christmas Island",
    "Cocos (Keeling) Islands",
    "Cook Islands",
    "Fiji",
    "French Polynesia",
    "Guam",
    "Heard Island and McDonald Islands",
    "Kiribati",
    "Marshall Islands",
    "Micronesia",
    "Nauru",
    "New Caledonia",
    "New Zealand",
    "Niue",
    "Norfolk Island",
    "Northern Mariana Islands",
    "Palau",
    "Papua New Guinea",
    "Pitcairn",
    "Samoa",
    "Solomon Islands",
    "Timor Gap",
    "Tokelau",
    "Tonga",
    "Tuvalu",
    "United States Minor Outlying Islands",
    "Vanuatu",
    "Venezuela-Trinidad and Tobago",
    "Wallis and Futuna"
  ]
}


current_countries_list = []

for v in current_countries.values():
    current_countries_list.extend(v)

print(current_countries_list)

print(current_countries_list)

['Algeria', 'Angola', 'Benin', 'Botswana', 'British Indian Ocean Territory', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cameroon', 'Central African Republic', 'Chad', 'Comoros', "Côte d'Ivoire", 'Djibouti', 'DR Congo', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'French Southern Territories', 'Gabon', 'The Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Mayotte', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Réunion', 'Rwanda', 'Saint Helena, Ascension, and Tristan da Cunha', 'Sao Tome and Principe', 'Senegal', 'Senegal-Mauritania', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Western Sahara', 'Zambia', 'Zimbabwe', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Aruba', 'Bahamas', 'Barbados', 'Belize', 'Bermuda', 'Bolivia', 'Bonaire, Sint Eustatius, and Saba', 'Bouvet Island', 'Braz

In [31]:
countries_list = list(countries)
countries_list = [c.replace('; ', '-') for c in countries_list]

missing_countries_areas = set(countries_list) - set(current_countries_list)

In [32]:
missing_countries_areas

{'Angola-Republic of the Congo',
 'Republic of the Congo',
 'Venezuela-Trinidad and Tobago'}

In [ ]:
# add missing_countries_areas to current_countries 
